`launch.json` 文件夹的路径配置
```json
${workspaceRoot} // 当前打开的文件夹的绝对路径

${workspaceRootFolderName} // 当前打开的文件夹的名字

${file} // 当前打开正在编辑的文件名，包括绝对路径，文件名，文件后缀名

${relativeFile} // 从当前打开的文件夹到当前打开的文件的路径

${fileBasename} // 当前打开的文件名+后缀名，不包括路径

${fileBasenameNoExtension} // 当前打开的文件的文件名，不包括路径和后缀名

${fileDirname} // 当前打开的文件所在的绝对路径，不包括文件名

${fileExtname} // 当前打开的文件的后缀名
```

# `data.py`

```python
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)
```
将文本中出现的所有 `word` 和它第一次出现时的次序 `idx` 存储

```python


class Corpus():

    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        assert os.path.exists(path)
        with open(path, 'r', encoding='utf8') as f:
            all_words = []
            for line in f.readlines():
                words = line.split() + ['<eos>']
                for word in words:
                    idx = self.dictionary.add_word(word)
                    all_words.append(idx)
        return torch.Tensor(all_words)
```

将整个 txt 文件的 `word` 映射为 `idx` 一维向量


# `model.py`
## `RNNModel`
```python
- 原始输入: idx 组成的时序张量 `[idx1, idx2, ..., idxt]`
- `self.encoder`: `self.encoder = nn.Embedding(ntoken, ninp)`: 将 `idx` embedding 到维度为 `ninp` 的向量空间
- `self.rnn`: 
    - LSTM: 使用`nn`里的`LSTM`:  `self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)`
    - GRU: 使用 `nn` 里的 `GRU`: `self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)`
    - RNN_TANH: 手动设置 `nonlinearity` 为 `tanh`, 调用 `nn` 里的 `RNN`: `self.rnn = nn.RNN(ninp, nhid, nlayers, dropout=dropout, nonlinearity=nonlinearity)`
    - RNN_RELU: 手动设置 `nonlinearity` 为 `relu`, 调用 `nn` 里的 `RNN`: `self.rnn = nn.RNN(ninp, nhid, nlayers, dropout=dropout, nonlinearity=nonlinearity)`

- `self.decoder`: `self.decoder = nn.Linear(ninp, ntoken)`, 使用一个全连接层将输出映射为 `ntoken` 维
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            # TODO: 这两个权重的维度应该是转置的关系吧
            self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        # TODO: 为什么不初始化 self.encoder.bias 是 Embedding 没有这个参数吗
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)
```

In [1]:
import torch
x = torch.arange(12).reshape(1, 3, 4)
len(x)

1

# `transformer`

